In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as po
import requests as rq

In [2]:
MAPBOX_TOKEN = 'pk.eyJ1IjoiZ3JvbDIwMjAiLCJhIjoiY2s4bnVxeDk0MTI5MDNqbzJ3N212d3JvNSJ9.g2Pe3QhqbcfGOedTchNgCw'

In [3]:
#read in latest info from https://github.com/datasets/covid-19
df = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv")
df.Date = pd.to_datetime(df.Date,format="%Y-%m-%d")
#create population column for later
df["Population"] = np.NaN
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN


In [152]:
cinfo = rq.get("https://raw.githubusercontent.com/gregory1506/DashPlotly/master/test.json").json()
df2 = df.copy()
df2["Region"] = np.nan
df2["Sub-Region"] = np.nan
df2["ISO3"] = np.nan
countries = df["Country/Region"].unique()
for country in countries:
    if country in cinfo:
        df2.loc[df2["Country/Region"] == country,["Population","Region","Sub-Region","ISO3"]] = cinfo[country][:]
df2.head(5)

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG


In [18]:
COUNTRIES = df2["Country/Region"].unique()
DATES = sorted(df2.Date.unique())

In [76]:
def mapbox_center(country=None):
    df_tmp = df2[df2["Country/Region"].isin(country)]
    lat = round(df_tmp.Lat.mean(),3)
    lon = round(df_tmp.Long.mean(),3)
    return dict(lat=lat,lon=lon)

def mapbox_zoom(country=None):
    if set(country) == set(COUNTRIES):
        return 1.07
    else:
        return 3.0

In [77]:
def make_covid_map(case=None,country=None,date=None):
        if country is None or country == []:
            country = COUNTRIES
        elif type(country) == str:
            country = list((country,))
        elif type(country) == list:
            pass
        else:
            print("Only lists or single string accepted for country")
        if date is None:
            date = DATES[-1]
        else:
            date = np.datetime64(date)
        df_tmp = df2[(df2.Date == date) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
        return {"data" : [go.Scattermapbox(
                                lat=df_tmp.Lat,
                                lon=df_tmp.Long,
                                mode="markers",
                                marker=go.scattermapbox.Marker(
                                        size=df_tmp.Confirmed ** (1/3),
                                        color="red",
                                        opacity=0.7
                                ),
                                text=df_tmp["Country/Region"] + " " + "Confirmed : " + df_tmp["Confirmed"].astype(str),
                                hoverinfo="text"
                                )
                        ],
                "layout" : go.Layout(
                                hovermode="closest",
                                autosize=True,
                                mapbox={"accesstoken":MAPBOX_TOKEN,
                                        "center":mapbox_center(country),
                                        "zoom":mapbox_zoom(country)
                                        },
                                )
        }
fig = go.Figure(make_covid_map(country=["Barbados","US"],date="2020-03-01"))
po.plot(fig,filename="mapbox2.html")

'mapbox2.html'

In [79]:
df2[(df2["Country/Region"] == "China") & (df2.Date == DATES[-1])]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3
4987,2020-04-16,China,Anhui,31.8257,117.2264,991.0,984.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5073,2020-04-16,China,Beijing,40.1824,116.4142,593.0,503.0,8.0,1.377422e+09,Asia,Eastern_Asia,CHN
5159,2020-04-16,China,Chongqing,30.0572,107.8740,579.0,570.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5245,2020-04-16,China,Fujian,26.0789,117.9874,353.0,333.0,1.0,1.377422e+09,Asia,Eastern_Asia,CHN
5331,2020-04-16,China,Gansu,37.8099,101.0583,139.0,137.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5417,2020-04-16,China,Guangdong,23.3417,113.4244,1571.0,1471.0,8.0,1.377422e+09,Asia,Eastern_Asia,CHN
5503,2020-04-16,China,Guangxi,23.8298,108.7881,254.0,252.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5589,2020-04-16,China,Guizhou,26.8154,106.8748,146.0,144.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5675,2020-04-16,China,Hainan,19.1959,109.7453,168.0,162.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5761,2020-04-16,China,Hebei,39.5490,116.1306,328.0,315.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN


In [57]:
mapbox_center(["US","Russia"])

{'lat': 48.545, 'lon': -2.856}

In [89]:
date = DATES[-1]
country = ["US","Canada"]
tmp = df2[(df2.Date == date) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
tmp1 = tmp.groupby(["Date"]).agg(sum).copy()
# tmp1["Confirmed"][0]
tmp1

,Lat,Long,Confirmed,Recovered,Deaths,Population
Date,,,,,,
2020-04-16,708.8509,-1333.3418,698610.0,54703.0,34174.0,793968331.0


In [96]:
date = DATES[-1]
country = ["US","Canada"]
tmp = df2[(df2.Date.isin(DATES[:DATES.index(date)])) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
tmp = tmp.groupby(["Date"]).agg(sum).copy()
fig = go.Figure()
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Deaths,fill='tozeroy',fillcolor="red",mode='none',name="Deaths"))
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Recovered,fill='tonexty',fillcolor="green",mode='none',name="Recovered"))
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Confirmed,fill='tonexty',fillcolor="blue",mode='none',name="Confirmed"))
fig.update_layout(showlegend=True,xaxis=dict(title="Progression of Outbreak (Date)"),yaxis=dict(title="Number of cases"))
po.plot(fig,filename="lineplot2.html")

'lineplot2.html'

In [92]:
datf = np.datetime64("2020-04-01")
DATES.index(datf)

70

In [131]:
cases = ["Confirmed","Recovered"]
COLORS = {"Confirmed":"red","Recovered":"green","Deaths":"blue"}
def make_covid_map(country=None,date=None,cases=None):
        if cases is None:
            cases = list(("Confirmed",))
        elif cases == "All":
            cases = ["Confirmed","Recovered","Deaths"]
        else:
            pass
        if country is None or country == []:
            country = COUNTRIES
        elif type(country) == str:
            country = list((country,))
        elif type(country) == list:
            pass
        else:
            print("Only lists or single string accepted for country")
        if date is None:
            date = DATES[-1]
        else:
            date = np.datetime64(date)
        fig = go.Figure()
        for case in cases:
                df_tmp = df2[(df2.Date == date) & (df2[case] > 0) & (df2["Country/Region"].isin(country))]
                fig.add_trace(go.Scattermapbox(
                                        lat=df_tmp.Lat,
                                        lon=df_tmp.Long,
                                        mode="markers",
                                        marker=go.scattermapbox.Marker(
                                                size=df_tmp[case] ** (1/3),
                                                color=COLORS[case],
                                                opacity=0.7
                                        ),
                                        text=df_tmp["Country/Region"] + "," +df_tmp["Province/State"] + f"<br>" "{case} : " + df_tmp[case].astype(str),
                                        hoverinfo="text"
                                        )
                                )
        fig.update_layout(hovermode="closest",
                autosize=False,
                mapbox={"accesstoken":MAPBOX_TOKEN,
                        "center":mapbox_center(country),
                        "zoom":mapbox_zoom(country)
                        },
                margin=dict(r=0,l=0,t=0,b=0,pad=0),
                legend={"y":0},
                showlegend=True
                )
        return fig

po.plot(make_covid_map(country=country,date=None,cases=None),filename="mapbox3.html")

'mapbox3.html'

In [134]:
df2["Sub-Region"].unique()

array(['Southern_Asia', 'Southern_Europe', 'Northern_Africa',
       'Middle_Africa', 'Caribbean', 'South_America', 'Western_Asia',
       'Australia_and_New_Zealand', 'Western_Europe', 'Eastern_Europe',
       'Central_America', 'Western_Africa', 'Southern_Africa',
       'South-Eastern_Asia', 'Eastern_Africa', 'Northern_America',
       'Eastern_Asia', 'Northern_Europe', nan, 'Melanesia',
       'Central_Asia'], dtype=object)

In [135]:
df2.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG


In [153]:
df2["DPM"] = round(((1000000 / df2.Population) * df.Deaths),3)

In [154]:
df2.DPM.max()

1151.341

In [155]:
df2.describe()

,Lat,Long,Confirmed,Recovered,Deaths,Population,DPM
count,22704.000000,22704.000000,22618.000000,21500.000000,22618.000000,2.253200e+04,22446.000000
mean,21.529941,21.765487,1554.115970,402.711721,84.698338,2.038236e+08,3.336960
std,24.745779,70.965413,15302.531159,3764.717215,944.873644,4.540415e+08,36.338292
min,-51.796300,-135.000000,-1.000000,0.000000,-1.000000,4.510000e+02,-0.028000
25%,7.405000,-27.932425,0.000000,0.000000,0.000000,4.741000e+06,0.000000
50%,23.659750,20.535638,2.000000,0.000000,0.000000,1.879920e+07,0.000000
75%,41.227200,78.750000,119.000000,11.000000,1.000000,6.511000e+07,0.002000
max,71.706900,178.065000,667801.000000,77000.000000,32916.000000,1.377422e+09,1151.341000


In [157]:
df2[df2.DPM > 100]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3,DPM
326,2020-03-30,Andorra,NaN,42.5063,1.5218,370.0,10.0,8.0,78014.0,Europe,Southern_Europe,AND,102.546
327,2020-03-31,Andorra,NaN,42.5063,1.5218,376.0,10.0,12.0,78014.0,Europe,Southern_Europe,AND,153.819
328,2020-04-01,Andorra,NaN,42.5063,1.5218,390.0,10.0,14.0,78014.0,Europe,Southern_Europe,AND,179.455
329,2020-04-02,Andorra,NaN,42.5063,1.5218,428.0,10.0,15.0,78014.0,Europe,Southern_Europe,AND,192.273
330,2020-04-03,Andorra,NaN,42.5063,1.5218,439.0,16.0,16.0,78014.0,Europe,Southern_Europe,AND,205.091
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21065,2020-04-12,United Kingdom,NaN,55.3781,-3.4360,84279.0,344.0,10612.0,65110000.0,Europe,Northern_Europe,GBR,162.986
21066,2020-04-13,United Kingdom,NaN,55.3781,-3.4360,88621.0,0.0,11329.0,65110000.0,Europe,Northern_Europe,GBR,173.998
21067,2020-04-14,United Kingdom,NaN,55.3781,-3.4360,93873.0,0.0,12107.0,65110000.0,Europe,Northern_Europe,GBR,185.947
21068,2020-04-15,United Kingdom,NaN,55.3781,-3.4360,98476.0,0.0,12868.0,65110000.0,Europe,Northern_Europe,GBR,197.635


In [166]:
tmp = df2[df2.Date == df2.Date.max()]
tmp = tmp.sort_values(by=["Confirmed"])[:10]
# fig = go.Figure([go.Bar(x=tmp["Country/Region"],y=tmp["Confirmed"])])
# po.plot(fig,filename="bar1.html")
tmp

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3,DPM
3611,2020-04-16,Canada,Diamond Princess,0.000000,0.000000,-1.0,NaN,-1.0,3.615549e+07,Americas,Northern_America,CAN,-0.028
7481,2020-04-16,China,Tibet,31.692700,88.092400,1.0,1.0,0.0,1.377422e+09,Asia,Eastern_Asia,CHN,0.000
22531,2020-04-16,Yemen,NaN,15.552727,48.516388,1.0,0.0,0.0,2.747800e+07,Asia,Western_Asia,YEM,0.000
10835,2020-04-16,France,Saint Pierre and Miquelon,46.885200,-56.315900,1.0,0.0,0.0,6.671000e+07,Europe,Western_Europe,FRA,0.000
21155,2020-04-16,United Kingdom,Anguilla,18.220600,-63.068600,3.0,1.0,0.0,6.511000e+07,Europe,Northern_Europe,GBR,0.000
21327,2020-04-16,United Kingdom,British Virgin Islands,18.420700,-64.640000,3.0,2.0,0.0,6.511000e+07,Europe,Northern_Europe,GBR,0.000
16167,2020-04-16,Netherlands,"Bonaire, Sint Eustatius and Saba",12.178400,-68.238500,3.0,0.0,0.0,1.701980e+07,Europe,Western_Europe,NLD,0.000
19349,2020-04-16,South Sudan,NaN,6.877000,31.307000,4.0,0.0,0.0,1.213100e+07,Africa,Middle_Africa,SSD,0.000
18403,2020-04-16,Sao Tome and Principe,NaN,0.186360,6.613081,4.0,0.0,0.0,1.873560e+05,Africa,Middle_Africa,STP,0.000
3095,2020-04-16,Burundi,NaN,-3.373100,29.918900,5.0,0.0,1.0,1.011450e+07,Africa,Eastern_Africa,BDI,0.099


'bar1.html'